<a href="https://colab.research.google.com/github/Mike030668/Spliter_Text2Movi_Kandinskiy_22/blob/main/notebooks/Constract_steps_cickle_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Mike030668/Spliter_Text2Movi_Kandinskiy_22.git -q

# Construct method to find points of start and end fragments movi

In [ ]:
import numpy as np
import torch
from torch import nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAIN_DIR =  "/content/Spliter_Text2Movi_Kandinskiy_22"
import sys
sys.path.append(MAIN_DIR)

## init rand data

In [ ]:
from utills.step_points import ComputeDiffPoints, Shuff_Reshuff

In [ ]:
cdp = ComputeDiffPoints()

# make set rand embeddings for tests
d_batch = 15
MAX_LEN_MOVI = 100

# sets images embeds
image_embeds = torch.randn(d_batch+1, 1, 1280)

# text embedings prompt video from Text model
text_hid_state = torch.randn((1, 77, 1280), requires_grad=True)

# unclip embedings prompt video from Prior
unclip_embed = torch.randn((1, 1, 1280), requires_grad=True)

# make set time_labels
labels = np.random.choice(np.arange(1, MAX_LEN_MOVI), d_batch+1, replace=False)

# start and end points to use for start normal and back ways
labels[0] = 0
labels[-1] = MAX_LEN_MOVI-1

print(labels)
cdp.time_labels = labels
config_norm, config_back = cdp.getpoints_train()

print(f"Time labels {labels}\n")
print('\nNorm_way')
print(f"Start base point {cdp.s_point}")
print(f"Base normal points {cdp.points}\n\n")
print('\nBack_way')
print(f"Start base back_point {cdp.back_s_point}")
print(f"Base back points {cdp.back_points}\n\n")

[ 0 15 11 38 17  1 47 76 16 27 29 89 40 20 12 99]
Time labels [ 0 15 11 38 17  1 47 76 16 27 29 89 40 20 12 99]


Norm_way
Start base point 0
Base normal points [15 11 38 17  1 47 76 16 27 29 89 40 20 12 99]



Back_way
Start base back_point 0
Base back points [87 79 59 10 70 72 83 23 52 98 82 61 88 84 99]




## compute normal and back points for rotation steps

In [ ]:
print(f"Time labels {labels}")
print('\n\nNorm rotation way')
print(f"Base normal points {cdp.points} for start rotation steps \n")
id_img_emb_s = config_norm["id_img_emb_s"]
print(f"ID image and text start rotation points {id_img_emb_s}")
id_uclip_emb = config_norm["id_uclip_emb"]
print(f"ID uclip_emb start rotation points {id_uclip_emb}")
id_img_delta = config_norm["id_img_delta"]
print(f"ID delta rotation points {id_img_delta}")
delta = config_norm["delta"]
print(f"Delta between points {delta} normal points")


print('\n\nNorm rotation way')
print(f"Base back points {cdp.back_points} for start rotation back steps \n")
id_img_emb_s = config_back["id_img_emb_s"]
print(f"ID image and text start rotation points {id_img_emb_s} in base back points")
id_uclip_emb = config_back["id_uclip_emb"]
print(f"ID uclip_emb start rotation points {id_uclip_emb} in base back points")
id_img_delta = config_back["id_img_delta"]
print(f"ID delta rotation points {id_img_delta} in base back points")
delta = config_back["delta"]
print(f"Delta between points {delta} back points")


Time labels [ 0 15 11 38 17  1 47 76 16 27 29 89 40 20 12 99]


Norm rotation way
Base normal points [15 11 38 17  1 47 76 16 27 29 89 40 20 12 99] for start rotation steps 

ID image and text start rotation points [1, 4, 13, 4, 0, 1, 3, 4, 7, 8, 9, 12, 13, 0, 1, 4, 7, 13, 0, 1, 2, 3, 4, 7, 8, 9, 11, 12, 13, 0, 1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 0, 1, 4, 13, 0, 1, 3, 4, 7, 12, 13, 0, 1, 3, 4, 7, 8, 12, 13, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 0, 1, 2, 3, 4, 7, 8, 9, 12, 13, 0, 1, 3, 4, 7, 13, 1, 4, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ID uclip_emb start rotation points [1, 4, 13, 4, 0, 1, 3, 4, 7, 8, 9, 12, 13, 0, 1, 4, 7, 13, 0, 1, 2, 3, 4, 7, 8, 9, 11, 12, 13, 0, 1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 0, 1, 4, 13, 0, 1, 3, 4, 7, 12, 13, 0, 1, 3, 4, 7, 8, 12, 13, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 0, 1, 2, 3, 4, 7, 8, 9, 12, 13, 0, 1, 3, 4, 7, 13, 1, 4, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ID delta rotation points [0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 

## compute normal and back points for diff steps

It will be used for prediction from prediction of model

In [ ]:
config_diff_norm, config_diff_back = cdp.getpoints_diftrain()

print(f"Time labels {labels}")
print('\n\nNorm diff way')
print(f"Base normal points {cdp.points}")
print(f"Next normal points {cdp.next_points} for diff steps \n")
id_img_emb_s = config_diff_norm["id_img_emb_s"]
print(f"ID image and text start diff points {id_img_emb_s} in base normal points")
id_uclip_emb = config_diff_norm["id_uclip_emb"]
print(f"ID uclip_emb start diff points {id_uclip_emb} in next normal points")
id_img_delta = config_diff_norm["id_img_delta"]
print(f"ID delta diff points {id_img_delta} in next normal points")
delta = config_diff_norm["delta"]
print(f"Delta between diff points {delta} in base normal points")


print('\n\nNorm rotation way')
print(f"Base back points {cdp.back_points}")
print(f"Next back points {cdp.back_next_points} for diff steps \n")
id_img_emb_s = config_diff_back["id_img_emb_s"]
print(f"ID image and text start diff points {id_img_emb_s} in base back points")
id_uclip_emb = config_diff_back["id_uclip_emb"]
print(f"ID uclip_emb start diff points {id_uclip_emb} in next back points")
id_img_delta = config_diff_back["id_img_delta"]
print(f"ID delta diff points {id_img_delta} in next back points")
delta = config_diff_back["delta"]
print(f"Delta between diff points {delta} back points")


Time labels [ 0 15 11 38 17  1 47 76 16 27 29 89 40 20 12 99]


Norm diff way
Base normal points [15 11 38 17  1 47 76 16 27 29 89 40 20 12 99]
Next normal points [ 16  12  39  18   2  48  77  17  28  30  90  41  21  13 100] for diff steps 

ID image and text start diff points [7, 13, 13] in base normal points
ID uclip_emb start diff points [0, 1, 1] in next normal points
ID delta diff points [3, 3, 7] in next normal points
Delta between diff points [1, 5, 4] in base normal points


Norm rotation way
Base back points [87 79 59 10 70 72 83 23 52 98 82 61 88 84 99]
Next back points [ 88  80  60  11  71  73  84  24  53  99  83  62  89  85 100] for diff steps 

ID image and text start diff points [6, 13, 6, 6, 12, 13] in base back points
ID uclip_emb start diff points [10, 6, 10, 10, 0, 6] in next back points
ID delta diff points [12, 12, 13, 14, 14, 14] in next back points
Delta between diff points [5, 4, 1, 16, 11, 15] back points


# Both ways steps

In [ ]:
from models.baseline import SpliterSimple
from utills.clearing import flush_memory

model = SpliterSimple(max_delta_time = 100,
                        emb_dim = 1280,
                        ways = 2
                        )

## Losses

nn.CosineEmbeddingLoss
https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html

In [ ]:
clip_text_last_hidden_state = torch.randn((2, 77, 1280)) #outputs.last_hidden_state
embeddings = torch.randn((2, 1, 1280))
delta = torch.tensor([[3],[5]])
direction  = torch.tensor([[0],[1]])
model.to(DEVICE)
out_embeddings = model(text_hidden_states = clip_text_last_hidden_state.to(DEVICE),
                       prior_embeds = embeddings.to(DEVICE),
                       delta_time = delta.to(DEVICE),
                       direction = direction.to(DEVICE)
                        )
out_embeddings.shape

torch.Size([2, 1, 1280])

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html
loss = nn.CosineEmbeddingLoss()
input1 = torch.randn(3, 1, requires_grad=True)
input2 = torch.randn(3, 5, requires_grad=True)
target = torch.ones(3)
output = loss(input1, input2, target)
output.backward()
output

tensor(0.5295, grad_fn=<MeanBackward0>)

## apply as vector losses

In [ ]:
mse_loss = nn.MSELoss(reduction='none').to(DEVICE)
cos_loss = nn.CosineEmbeddingLoss(reduction = 'none').to(DEVICE)
target = torch.ones(1280).to(DEVICE)
loss_cos = cos_loss(embeddings.squeeze(1).T.to(DEVICE), out_embeddings.squeeze(1).T, target)
print(loss_cos)

tensor([1.7729, 0.5990, 0.0446,  ..., 1.2958, 0.9652, 0.7654], device='cuda:0',
       grad_fn=<AddBackward0>)


## first steps normal and back shuffleed

In [ ]:
all_points = np.append(cdp.points, cdp.back_points)
# intit class for shufflee
b_srs = Shuff_Reshuff(len(all_points))
bs_idx = b_srs.shuffle() # shuffleed indexees
bu_idx = b_srs.unshuffle() # indexees for re_shuffle

B_batch = 2*d_batch

time_base = torch.tensor(all_points).unsqueeze(1)
directions = torch.concat([torch.zeros_like(time_base[:d_batch]), torch.ones_like(time_base[d_batch:])])

# shuffleed
time_base = time_base.to(DEVICE)
directions = directions.to(DEVICE)


text_hid_states =  torch.concat([text_hid_state for _ in range(B_batch)])
base_unclip_embs = torch.concat([unclip_embed for _ in range(B_batch)])

base_img_embs = [image_embeds[0] for _ in range(d_batch)] + [image_embeds[-1] for _ in range(d_batch)]
base_img_embs = torch.concat(base_img_embs)


back_img_embs = torch.concat([image_embeds[i,:,:] for i in range(image_embeds.shape[0])][::-1]).unsqueeze(1)
img_embs =[image_embeds[1:,:,:], back_img_embs[1:,:,:]]
img_embs = torch.concat(img_embs)


pred_unclip_embs = model(
      text_hidden_states = text_hid_states[bs_idx].to(torch.float32).to(DEVICE), # shuffleed
      prior_embeds = base_unclip_embs[bs_idx].to(torch.float32).to(DEVICE),
      delta_time = time_base[bs_idx],
      direction = directions[bs_idx]
                        )

# difference
diff_img_embs =  (base_img_embs[bs_idx].squeeze(dim=1) - img_embs[bs_idx].squeeze(dim=1)) #
diff_unclip_embs = (base_unclip_embs[bs_idx].squeeze(dim=1).to(DEVICE) - pred_unclip_embs.squeeze(dim=1))

# CosineEmbeddingLoss between difference
movi_cos_loss = cos_loss(diff_unclip_embs.T, diff_img_embs.T.to(DEVICE), target).half()  #

# MSELoss between predict and
movi_mse_loss = mse_loss(diff_unclip_embs.half(), diff_img_embs.half().to(DEVICE))#

movi_loss = movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

del(time_base, directions, diff_img_embs, diff_unclip_embs)
flush_memory()

print(movi_loss)

tensor([2.5410, 2.2637, 1.9648,  ..., 2.2578, 4.3359, 2.6211], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddBackward0>)


## Rotation steps normal and back shuffleed

In [ ]:
from utills.rotations import RotationVectors

In [ ]:
# inite rotation class
RV = RotationVectors()

In [ ]:
rote_norm_train = len(config_norm['id_uclip_emb'])
rote_back_train = len(config_back['id_uclip_emb'])
rote_train =  rote_norm_train + rote_back_train

if rote_train:
    text_hid_states_2rt = []
    unclip_embs_2rt = []
    base_img_embs_2rt = []
    image_embs_2rt = []
    delta_2rt = []
    direction_2rt = []

    # intit class for shufflee again
    srs = Shuff_Reshuff(rote_train)
    s_idx = srs.shuffle() # shuffleed indexees
    u_idx = srs.unshuffle() # indexees for re_shuffle


    if rote_norm_train:
        # collect norm steps
        text_hid_states_2rt.append(torch.clone(text_hid_states[:d_batch])[config_norm['id_uclip_emb']])
        unclip_embs_2rt.append(torch.clone(base_unclip_embs[:d_batch])[config_norm['id_uclip_emb']])
        base_img_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_emb_s']])
        image_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_delta']])
        delta_2rt.append(torch.tensor(config_norm['delta']).unsqueeze(1))
        direction_2rt.append(torch.zeros_like(delta_2rt[-1]))


    if rote_back_train:
        # collect back steps
        text_hid_states_2rt.append(torch.clone(text_hid_states[d_batch:])[config_back['id_uclip_emb']])
        unclip_embs_2rt.append(torch.clone(base_unclip_embs[d_batch:])[config_back['id_uclip_emb']])
        base_img_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_emb_s']])
        image_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_delta']])
        delta_2rt.append(torch.tensor(config_back['delta']).unsqueeze(1))
        direction_2rt.append(torch.ones_like(delta_2rt[-1]))

    # concat tansors
    if rote_train > B_batch:
       R_batch = B_batch
    else: R_batch = rote_train

    # shufle and take R_batch samples
    text_hid_states_2rt = torch.concat(text_hid_states_2rt)[s_idx][:R_batch].to(DEVICE)
    unclip_embs_2rt = torch.concat(unclip_embs_2rt)[s_idx][:R_batch].to(DEVICE)
    base_img_embs_2rt = torch.concat(base_img_embs_2rt)[s_idx][:R_batch].to(DEVICE)
    image_embs_2rt = torch.concat(image_embs_2rt)[s_idx][:R_batch].to(DEVICE)
    delta_2rt = torch.concat(delta_2rt)[s_idx][s_idx][:R_batch].to(DEVICE)
    direction_2rt = torch.concat(direction_2rt)[s_idx][:R_batch].to(DEVICE)

    # get rotation vectors
    R_marixes = RV.get_rotation_matrix(base_img_embs_2rt.squeeze(1), image_embs_2rt.squeeze(1)).to(DEVICE)
    text_hid_states_2rt = torch.bmm(text_hid_states_2rt, R_marixes)
    unclip_embs_2rt = torch.bmm(unclip_embs_2rt, R_marixes)

    pred_rote_embs = model(
                  text_hidden_states = text_hid_states_2rt.to(torch.float32), # shuffleed
                  prior_embeds = unclip_embs_2rt.to(torch.float32),
                  delta_time = delta_2rt,
                  direction = direction_2rt
                                    )

    # difference
    diff_unclip_embeds = (unclip_embs_2rt.squeeze(dim=1) - pred_rote_embs.squeeze(dim=1))
    diff_img_embs =  (base_img_embs_2rt.squeeze(dim=1) - image_embs_2rt.squeeze(dim=1))

    # CosineEmbeddingLoss between difference
    movi_cos_loss = cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  # .nan_to_num()

    # MSELoss between predict and
    movi_mse_loss = mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)) #

    movi_loss = movi_loss + movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

    del(delta_2rt, direction_2rt, diff_img_embs, diff_unclip_embeds)
    del(pred_rote_embs, text_hid_states_2rt, R_marixes)
    del(unclip_embs_2rt,image_embs_2rt)
    flush_memory()


    print(movi_loss)




tensor([5.4102, 4.2930, 5.7227,  ..., 5.4375, 7.8203, 5.3281], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddBackward0>)


## Next steps normal and back from pred steps shuffleed

In [ ]:
diff_norm = len(config_diff_norm['id_uclip_emb'])
diff_back = len(config_diff_back['id_uclip_emb'])
diff_train = diff_norm + diff_back

if diff_train:
    take_text_hid_states = []
    take_base_unclip_embs = []
    next_unclip_embs = []
    next_base_img_embs = []
    next_image_embs = []
    next_delta = []
    next_text_hid_states = []
    next_direction = []

    # un_shuffleed
    pred_unclip_embs = torch.clone(pred_unclip_embs.detach().cpu())[bu_idx]

    # intit class for shufflee again
    srs = Shuff_Reshuff(diff_train)
    s_idx = srs.shuffle() # shuffleed indexees
    u_idx = srs.unshuffle() # indexees for re_shuffle

    if diff_norm:
        # collect next norm steps
        take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
        take_text_hid_states.append(torch.clone(text_hid_states[:d_batch])[config_diff_norm['id_uclip_emb']])
        next_unclip_embs.append(torch.clone(pred_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
        next_base_img_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_emb_s']])
        next_image_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_delta']])
        next_delta.append(torch.tensor(config_diff_norm['delta']).unsqueeze(1))
        next_direction.append(torch.zeros_like(next_delta[-1]))

    if diff_back:
        # collect next back steps
        take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_back['id_uclip_emb']])
        take_text_hid_states.append(torch.clone(text_hid_states[d_batch:])[config_diff_back['id_uclip_emb']])
        next_unclip_embs.append(torch.clone(pred_unclip_embs[d_batch:])[config_diff_back['id_uclip_emb']])
        next_base_img_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_emb_s']])
        next_image_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_delta']])
        next_delta.append(torch.tensor(config_diff_back['delta']).unsqueeze(1))
        next_direction.append(torch.ones_like(next_delta[-1]))



    take_base_unclip_embs = torch.concat(take_base_unclip_embs).to(DEVICE)
    take_text_hid_states = torch.concat(take_text_hid_states).to(DEVICE)
    next_unclip_embs = torch.concat(next_unclip_embs).to(DEVICE)
    next_base_img_embs = torch.concat(next_base_img_embs)
    next_image_embs = torch.concat(next_image_embs)
    next_delta = torch.concat(next_delta)[s_idx].to(DEVICE)
    next_direction = torch.concat(next_direction)[s_idx].to(DEVICE)

    # get rotation vectors
    R_marixes = RV.get_rotation_matrix(take_base_unclip_embs.squeeze(1), next_unclip_embs.squeeze(1))
    next_text_hid_states = torch.bmm(take_text_hid_states, R_marixes)


    next_pred_unclip_embs = model(
        text_hidden_states = next_text_hid_states[s_idx].to(torch.float32),
        prior_embeds = next_unclip_embs[s_idx].to(torch.float32),
        delta_time = next_delta,
        direction = next_direction
        )

    # difference
    diff_unclip_embeds = (next_unclip_embs[s_idx].squeeze(dim=1).to(DEVICE) - next_pred_unclip_embs.squeeze(dim=1))
    diff_img_embs =  (next_base_img_embs[s_idx].squeeze(dim=1) - next_image_embs[s_idx].squeeze(dim=1))

    # CosineEmbeddingLoss between difference
    movi_cos_loss = cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  # .nan_to_num()

    # MSELoss between predict and
    movi_mse_loss = mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)) #

    next_movi_loss = movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

    del(diff_img_embs, diff_unclip_embeds)
    del(next_image_embs, next_base_img_embs, next_delta, next_direction)
    del(next_text_hid_states,  next_unclip_embs)
    del(pred_unclip_embs, next_pred_unclip_embs)
    flush_memory()

    print(next_movi_loss)

tensor([4.7500, 1.7344, 3.4199,  ..., 0.8779, 1.7559, 2.7773], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddBackward0>)


##  Cicle

### init rand data

In [ ]:
all_data = []
# make set rand embeddings for tests
d_batch = 15
q_ty = 100

for i in range(q_ty):
    data = dict()
    # make set time_labels
    labels = np.random.choice(np.arange(1, 100), d_batch+1, replace=False)
    labels[0] = 0
    labels[-1] = 100
    data['labels'] = labels

    data['image_embeds'] = []
    for i in range(d_batch+1):
      if not i : data['image_embeds'].append(torch.randn(1, 1, 1280))
      else:
        next = data['image_embeds'][-1] + 0.0001*torch.randn(1, 1, 1280)
        data['image_embeds'].append(next)

    data['text_hid_state'] = torch.randn((1, 77, 1280))
    data['unclip_embed'] = torch.randn((1, 1, 1280), requires_grad=True)

    all_data.append(data)


In [ ]:

LR_RATE = 1e-03
EPS = 0.95

mse_loss = nn.MSELoss(reduction='none').to(DEVICE)
cos_loss = nn.CosineEmbeddingLoss(reduction = 'none').to(DEVICE)
target = torch.ones(1280).to(DEVICE)


optimizer = torch.optim.AdamW(model.parameters(), lr=LR_RATE)

# https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                              cycle_momentum = False,
                                              base_lr=LR_RATE/10,
                                              max_lr=LR_RATE,
                                              step_size_up=5,
                                              mode="triangular2")
model.train()

# inite rotation class
RV = RotationVectors()

### test train

In [ ]:
# отрисовка прохождения цикла
from tqdm.notebook import tqdm
from random import shuffle
import time

step = 0
EPOHS = 10

for ep  in tqdm(range(EPOHS), unit ="EPOHS",
                      desc ="Пробегаемся по всем эпохам"):

    shuffle(all_data)
    for movi in all_data:

        optimizer.zero_grad()
        # make set rand embeddings for tests
        image_embeds = torch.concat(movi['image_embeds'])
        text_hid_state = movi['text_hid_state']
        unclip_embed = movi['unclip_embed']
        # make set time_labels
        labels = movi['labels']

        # place labels to class points
        cdp.time_labels = labels
        all_points = np.append(cdp.points, cdp.back_points)

        # intit class for shufflee
        b_srs = Shuff_Reshuff(len(all_points))
        bs_idx = b_srs.shuffle() # shuffleed indexees
        bu_idx = b_srs.unshuffle() # indexees for re_shuffle

        B_batch = 2*d_batch

        # collect time directions tensors
        time_base = torch.tensor(all_points).unsqueeze(1)
        directions = torch.concat([torch.zeros_like(time_base[:d_batch]),
                                   torch.ones_like(time_base[d_batch:])])
        # shuffleed
        time_base = time_base.to(DEVICE)
        directions = directions.to(DEVICE)

        # collect text and unclip tensors
        text_hid_states =  torch.concat([text_hid_state for _ in range(B_batch)])
        base_unclip_embs = torch.concat([unclip_embed for _ in range(B_batch)])

        # collect base_img_embs tensors
        base_img_embs = [image_embeds[0] for _ in range(d_batch)] \
        + [image_embeds[-1] for _ in range(d_batch)]
        base_img_embs = torch.concat(base_img_embs)

        # collect back_img_embs tensors
        back_img_embs = torch.concat([image_embeds[i,:,:] for i in range(image_embeds.shape[0])][::-1]).unsqueeze(1)
        # collect img_embs together tensors
        img_embs =[image_embeds[1:,:,:], back_img_embs[1:,:,:]]
        img_embs = torch.concat(img_embs)

        # base predict which can used in next step
        pred_unclip_embs = model(
              text_hidden_states = text_hid_states[bs_idx].to(torch.float32).to(DEVICE), # shuffleed
              prior_embeds = base_unclip_embs[bs_idx].to(torch.float32).to(DEVICE),
              delta_time = time_base[bs_idx],
              direction = directions[bs_idx]
                                )

        # difference
        diff_img_embs = (base_img_embs[bs_idx].squeeze(dim=1) - img_embs[bs_idx].squeeze(dim=1)) #
        diff_unclip_embs = (base_unclip_embs[bs_idx].squeeze(dim=1).to(DEVICE) - pred_unclip_embs.squeeze(dim=1))

        # CosineEmbeddingLoss between difference
        movi_cos_loss = cos_loss(diff_unclip_embs.T, diff_img_embs.T.to(DEVICE), target).half()  #

        # MSELoss between predict and
        movi_mse_loss = mse_loss(diff_unclip_embs.half(), diff_img_embs.half().to(DEVICE))#

        movi_loss = movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

        del(time_base, directions, diff_img_embs, diff_unclip_embs)


        # Rotation train steps
        rote_norm_train = len(config_norm['id_uclip_emb'])
        rote_back_train = len(config_back['id_uclip_emb'])
        rote_train =  rote_norm_train + rote_back_train

        if rote_train:
            text_hid_states_2rt = []
            unclip_embs_2rt = []
            base_img_embs_2rt = []
            image_embs_2rt = []
            delta_2rt = []
            direction_2rt = []

            # intit class for shufflee again
            srs = Shuff_Reshuff(rote_train)
            s_idx = srs.shuffle() # shuffleed indexees
            u_idx = srs.unshuffle() # indexees for re_shuffle


            if rote_norm_train:
                # collect norm steps
                text_hid_states_2rt.append(torch.clone(text_hid_states[:d_batch])[config_norm['id_uclip_emb']])
                unclip_embs_2rt.append(torch.clone(base_unclip_embs[:d_batch])[config_norm['id_uclip_emb']])
                base_img_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_emb_s']])
                image_embs_2rt.append(torch.clone(img_embs[:d_batch])[config_norm['id_img_delta']])
                delta_2rt.append(torch.tensor(config_norm['delta']).unsqueeze(1))
                direction_2rt.append(torch.zeros_like(delta_2rt[-1]))


            if rote_back_train:
                # collect back steps
                text_hid_states_2rt.append(torch.clone(text_hid_states[d_batch:])[config_back['id_uclip_emb']])
                unclip_embs_2rt.append(torch.clone(base_unclip_embs[d_batch:])[config_back['id_uclip_emb']])
                base_img_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_emb_s']])
                image_embs_2rt.append(torch.clone(img_embs[d_batch:])[config_back['id_img_delta']])
                delta_2rt.append(torch.tensor(config_back['delta']).unsqueeze(1))
                direction_2rt.append(torch.ones_like(delta_2rt[-1]))

            # concat tansors
            if rote_train > B_batch:
              R_batch = B_batch
            else: R_batch = rote_train

            # shufle and take R_batch samples
            text_hid_states_2rt = torch.concat(text_hid_states_2rt)[s_idx][:R_batch].to(DEVICE)
            unclip_embs_2rt = torch.concat(unclip_embs_2rt)[s_idx][:R_batch].to(DEVICE)
            base_img_embs_2rt = torch.concat(base_img_embs_2rt)[s_idx][:R_batch].to(DEVICE)
            image_embs_2rt = torch.concat(image_embs_2rt)[s_idx][:R_batch].to(DEVICE)
            delta_2rt = torch.concat(delta_2rt)[s_idx][s_idx][:R_batch].to(DEVICE)
            direction_2rt = torch.concat(direction_2rt)[s_idx][:R_batch].to(DEVICE)

            # get rotation vectors
            R_marixes = RV.get_rotation_matrix(base_img_embs_2rt.squeeze(1), image_embs_2rt.squeeze(1)).to(DEVICE)
            text_hid_states_2rt = torch.bmm(text_hid_states_2rt, R_marixes)
            unclip_embs_2rt = torch.bmm(unclip_embs_2rt, R_marixes)

            # rotation predict
            pred_rote_embs = model(
                          text_hidden_states = text_hid_states_2rt.to(torch.float32),
                          prior_embeds = unclip_embs_2rt.to(torch.float32),
                          delta_time = delta_2rt,
                          direction = direction_2rt
                                            )

            # difference
            diff_unclip_embeds = (unclip_embs_2rt.squeeze(dim=1) - pred_rote_embs.squeeze(dim=1))
            diff_img_embs =  (base_img_embs_2rt.squeeze(dim=1) - image_embs_2rt.squeeze(dim=1))

            # CosineEmbeddingLoss between difference
            movi_cos_loss = cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  #

            # MSELoss between predict and
            movi_mse_loss = mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)) #

            movi_loss = movi_loss + movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

            del(delta_2rt, direction_2rt, diff_img_embs, diff_unclip_embeds)
            del(pred_rote_embs, text_hid_states_2rt, R_marixes)
            del(unclip_embs_2rt,image_embs_2rt)

        # Diff train steps
        diff_norm = len(config_diff_norm['id_uclip_emb'])
        diff_back = len(config_diff_back['id_uclip_emb'])
        diff_train = diff_norm + diff_back

        if diff_train:
            take_text_hid_states = []
            take_base_unclip_embs = []
            next_unclip_embs = []
            next_base_img_embs = []
            next_image_embs = []
            next_delta = []
            next_text_hid_states = []
            next_direction = []

            # un_shuffleed
            pred_unclip_embs = torch.clone(pred_unclip_embs.detach().cpu())[bu_idx]

            # intit class for shufflee again
            srs = Shuff_Reshuff(diff_train)
            s_idx = srs.shuffle() # shuffleed indexees
            u_idx = srs.unshuffle() # indexees for re_shuffle

            if diff_norm:
                # collect next norm steps
                take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
                take_text_hid_states.append(torch.clone(text_hid_states[:d_batch])[config_diff_norm['id_uclip_emb']])
                next_unclip_embs.append(torch.clone(pred_unclip_embs[:d_batch])[config_diff_norm['id_uclip_emb']])
                next_base_img_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_emb_s']])
                next_image_embs.append(torch.clone(img_embs[:d_batch])[config_diff_norm['id_img_delta']])
                next_delta.append(torch.tensor(config_diff_norm['delta']).unsqueeze(1))
                next_direction.append(torch.zeros_like(next_delta[-1]))

            if diff_back:
                # collect next back steps
                take_base_unclip_embs.append(torch.clone(base_unclip_embs[:d_batch])[config_diff_back['id_uclip_emb']])
                take_text_hid_states.append(torch.clone(text_hid_states[d_batch:])[config_diff_back['id_uclip_emb']])
                next_unclip_embs.append(torch.clone(pred_unclip_embs[d_batch:])[config_diff_back['id_uclip_emb']])
                next_base_img_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_emb_s']])
                next_image_embs.append(torch.clone(img_embs[d_batch:])[config_diff_back['id_img_delta']])
                next_delta.append(torch.tensor(config_diff_back['delta']).unsqueeze(1))
                next_direction.append(torch.ones_like(next_delta[-1]))



            take_base_unclip_embs = torch.concat(take_base_unclip_embs).to(DEVICE)
            take_text_hid_states = torch.concat(take_text_hid_states).to(DEVICE)
            next_unclip_embs = torch.concat(next_unclip_embs).to(DEVICE)
            next_base_img_embs = torch.concat(next_base_img_embs)
            next_image_embs = torch.concat(next_image_embs)
            next_delta = torch.concat(next_delta)[s_idx].to(DEVICE)
            next_direction = torch.concat(next_direction)[s_idx].to(DEVICE)

            # get rotation vectors
            R_marixes = RV.get_rotation_matrix(take_base_unclip_embs.squeeze(1), next_unclip_embs.squeeze(1))
            next_text_hid_states = torch.bmm(take_text_hid_states, R_marixes)

            # dif predict from base predict
            next_pred_unclip_embs = model(
                text_hidden_states = next_text_hid_states[s_idx].to(torch.float32),
                prior_embeds = next_unclip_embs[s_idx].to(torch.float32),
                delta_time = next_delta,
                direction = next_direction
                )

            # difference
            diff_unclip_embeds = (next_unclip_embs[s_idx].squeeze(dim=1).to(DEVICE) - next_pred_unclip_embs.squeeze(dim=1))
            diff_img_embs =  (next_base_img_embs[s_idx].squeeze(dim=1) - next_image_embs[s_idx].squeeze(dim=1))

            # CosineEmbeddingLoss between difference
            movi_cos_loss = cos_loss(diff_unclip_embeds.T, diff_img_embs.T.to(DEVICE), target).half()  # .nan_to_num()

            # MSELoss between predict and
            movi_mse_loss = mse_loss(diff_unclip_embeds.half(), diff_img_embs.half().to(DEVICE)) #

            # collect loss
            movi_loss = movi_loss + movi_cos_loss + movi_mse_loss.mean(0) # (0) for 1280

            del(diff_img_embs, diff_unclip_embeds)
            del(next_image_embs, next_base_img_embs, next_delta, next_direction)
            del(next_text_hid_states,  next_unclip_embs)
            del(pred_unclip_embs, next_pred_unclip_embs)


    movi_loss.backward(torch.ones_like(movi_loss))

    optimizer.step()
    scheduler.step()

    #flush_memory()
    cur_lr = scheduler.get_last_lr()[0]
    cur_lr = round(cur_lr, 5)

    print(f'ep {ep} movi_loss {movi_loss.mean().item()/len(all_data)} with lr {cur_lr}', end ='\r')
    print()
    time.sleep(1)

    # https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling
    LR_RATE*=0.97
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                                  cycle_momentum = False,
                                                  base_lr=LR_RATE/10,
                                                  max_lr=LR_RATE,
                                                  step_size_up=5,
                                                  mode="triangular2")

Пробегаемся по всем эпохам:   0%|          | 0/10 [00:00<?, ?EPOHS/s]

ep 0 movi_loss 0.05015625 with lr 0.00028
ep 1 movi_loss 0.0512890625 with lr 0.00027
ep 2 movi_loss 0.0525 with lr 0.00026
ep 3 movi_loss 0.05078125 with lr 0.00026
ep 4 movi_loss 0.049921875 with lr 0.00025
ep 5 movi_loss 0.0497265625 with lr 0.00024
ep 6 movi_loss 0.050703125 with lr 0.00023
ep 7 movi_loss 0.050390625 with lr 0.00023
ep 8 movi_loss 0.05125 with lr 0.00022
ep 9 movi_loss 0.0522265625 with lr 0.00021
